In [1]:
# Python version must be 3.6 ~ 3.10, which is officially supported for pyrealsense2 library
!pip install -r requirements.txt

In [2]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import glob

In [3]:
IMAGE_CAPTURE_D435I_PATH = "captured_images/D435i/"

In [40]:
# === Input image path ===
input_path = "testImage_2.png" 
output_path = "testImage_2_annotated.png"

# === Load and preprocess image ===
image = cv2.imread(input_path)
if image is None:
    raise FileNotFoundError(f"Image not found at: {input_path}")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# === Enhance contrast (CLAHE) ===
clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
gray_eq = clahe.apply(gray)

# === Threshold to extract dark regions (potential fractures) ===
_, binary = cv2.threshold(gray_eq, 85, 255, cv2.THRESH_BINARY_INV)

# === Morphological cleaning ===
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

# === Find contours ===
contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# === Filter and draw yellow bounding boxes ===
output = image.copy()
fracture_count = 0

for cnt in contours:
    area = cv2.contourArea(cnt)
    if 10 < area < 1000:  # Tune this range as needed
        x, y, w, h = cv2.boundingRect(cnt)

        # Optional: Filter by shape (circularity)
        perimeter = cv2.arcLength(cnt, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * area / (perimeter ** 2)
        if circularity > 0.3:
            cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 255), 2)  # Yellow box
            cv2.putText(output, "Fracture", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 255), 1)  # Yellow text
            fracture_count += 1

# === Save and show result ===
cv2.imwrite(output_path, output)
print(f"Fracture detection complete. {fracture_count} regions identified.")
print(f"Annotated image saved to: {output_path}")

cv2.imshow("Fracture Detection (Yellow)", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


Fracture detection complete. 3 regions identified.
Annotated image saved to: testImage_2_annotated.png
